In [ ]:
import pandas as pd
import zarr
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

metadata = pd.read_csv("acquisitions_metadata.csv")
metadata

In [ ]:
pixelmet = pd.read_hdf("metadata.h5ad", key="table")
pixelmet

In [ ]:
import re
filez = [
 "composite_transposed_['Female1']_[5.]_[74.].tif",
 "composite_transposed_['Female1']_[1.]_[75.].tif",
 "composite_transposed_['Female1']_[2.]_[76.].tif",
 "composite_transposed_['Female1']_[3.]_[77.].tif",
 "composite_transposed_['Female1']_[4.]_[78.].tif",
 "composite_transposed_['Female3']_[1.]_[84.].tif",
 "composite_transposed_['Female3']_[2.]_[85.].tif",
 "composite_transposed_['Female3']_[6.]_[86.].tif",
 "composite_transposed_['Female3']_[3.]_[87.].tif",
 "composite_transposed_['Female3']_[4.]_[88.].tif",
 "composite_transposed_['Female3']_[5.]_[89.].tif",
 "composite_transposed_['Pregnant1']_[1.]_[108.].tif",
 "composite_transposed_['Pregnant1']_[2.]_[109.].tif",
 "composite_transposed_['Pregnant1']_[3.]_[110.].tif",
 "composite_transposed_['Pregnant1']_[4.]_[111.].tif",
 "composite_transposed_['Pregnant1']_[5.]_[112.].tif",
 "composite_transposed_['Pregnant1']_[6.]_[113.].tif",
 "composite_transposed_['Pregnant2']_[1.]_[114.].tif",
 "composite_transposed_['Pregnant2']_[2.]_[115.].tif",
 "composite_transposed_['Pregnant2']_[3.]_[116.].tif",
 "composite_transposed_['Pregnant2']_[4.]_[117.].tif",
 "composite_transposed_['Pregnant2']_[5.]_[118.].tif",
 "composite_transposed_['Pregnant2']_[6.]_[119.].tif",
 "composite_transposed_['Pregnant4']_[1.]_[120.].tif",
 "composite_transposed_['Pregnant4']_[2.]_[121.].tif",
 "composite_transposed_['Pregnant4']_[3.]_[122.].tif",
 "composite_transposed_['Pregnant4']_[4.]_[123.].tif",
 "composite_transposed_['Pregnant4']_[5.]_[124.].tif",
 "composite_transposed_['Pregnant4']_[6.]_[125.].tif"]

extracted = [re.search(r"_\['(.*?)'\]", file).group(1) for file in filez]
extracted_section_numbers = [re.search(r"_\[[^\]]*\]_\[[^\]]*\]_\[(.*?)\]", file).group(1) for file in filez]

In [ ]:
# for the females and pregnant females

import re
import os
import skimage
import numpy as np
import pandas as pd
import skimage.io


SAMPLETOADD = ['Female1', 'Female3', 'Pregnant1', 'Pregnant2', 'Pregnant4']
ccfs = []

for SAMPLENOW in SAMPLETOADD:

    filezNOW = np.array(filez)[np.array(extracted) == SAMPLENOW]
    sectionsNOW = np.array(extracted_section_numbers)[np.array(extracted) == SAMPLENOW]

    dfs = []
    
    for it, yyy in zip(filezNOW, sectionsNOW):
        yyy = str(int(float(yyy)))
        print(yyy)
        
        try:

            image = skimage.io.imread(f"/data/luca/lipidatlas/ManuscriptAnalysisRound3/annotation/"+SAMPLENOW+"/"+it+"_Coords.tif")
            height, width, _ = image.shape
            x_coords, y_coords = np.meshgrid(np.arange(width), np.arange(height))
            x_coords_flat = x_coords.flatten()
            y_coords_flat = y_coords.flatten()
            flattened_array = image.reshape(-1, 3)

            df = pd.DataFrame(flattened_array, columns=['xccf', 'yccf', 'zccf'])

            df['x_pos'] = x_coords_flat
            df['y_pos'] = y_coords_flat
            
            df.index = "section"+yyy+"_pixel"+(df['y_pos']+1).astype(str)+"_"+(df['x_pos']+1).astype(str)
            df.drop(columns=['x_pos', 'y_pos'], inplace=True)

            dfs.append(df)

        except:
            print("issue with: " + it)
        
    this_ccf = pd.concat(dfs)

    indexes = np.array(this_ccf.index)
    pixels_index_set = set(pixelmet.index)
    goodindexes = [ind for ind in indexes if ind in pixels_index_set]
    this_ccf = this_ccf.loc[goodindexes,:]
    this_ccf['x_index'] = (this_ccf['xccf']*40).astype(int)
    this_ccf['y_index'] = (this_ccf['yccf']*40).astype(int)
    this_ccf['z_index'] = (this_ccf['zccf']*40).astype(int)
    print(this_ccf)
    ccfs.append(this_ccf)

In [ ]:
# for the last female

import re
import os
import skimage
import numpy as np
import pandas as pd
import skimage.io

filezNOW = ["composite_transposed_['female214']_['A']_['female'].tif",
 "composite_transposed_['female214']_['F']_['female'].tif",
 "composite_transposed_['female214']_['C']_['female'].tif",
 "composite_transposed_['female214']_['E']_['female'].tif",
 "composite_transposed_['female214']_['D']_['female'].tif"]
sectionsNOW = [79.0, 80.0, 81.0, 82.0, 83.0]

dfs = []

for it, yyy in zip(filezNOW, sectionsNOW):
    yyy = str(int(float(yyy)))
    print(yyy)

    try:

        image = skimage.io.imread(f"/data/luca/lipidatlas/ManuscriptAnalysisRound3/annotation/female_214_exported/"+it+"_Coords.tif")
        height, width, _ = image.shape
        x_coords, y_coords = np.meshgrid(np.arange(width), np.arange(height))
        x_coords_flat = x_coords.flatten()
        y_coords_flat = y_coords.flatten()
        flattened_array = image.reshape(-1, 3)

        df = pd.DataFrame(flattened_array, columns=['xccf', 'yccf', 'zccf'])

        df['x_pos'] = x_coords_flat
        df['y_pos'] = y_coords_flat

        df.index = "section"+yyy+"_pixel"+(df['y_pos']+1).astype(str)+"_"+(df['x_pos']+1).astype(str)
        df.drop(columns=['x_pos', 'y_pos'], inplace=True)

        dfs.append(df)

    except:
        print("issue with: " + it)

this_ccf = pd.concat(dfs)

indexes = np.array(this_ccf.index)
pixels_index_set = set(pixelmet.index)
goodindexes = [ind for ind in indexes if ind in pixels_index_set]
this_ccf = this_ccf.loc[goodindexes,:]
this_ccf['x_index'] = (this_ccf['xccf']*40).astype(int)
this_ccf['y_index'] = (this_ccf['yccf']*40).astype(int)
this_ccf['z_index'] = (this_ccf['zccf']*40).astype(int)
print(this_ccf)
ccfs.append(this_ccf)

In [ ]:
# for the males

import re
import os
import skimage
import numpy as np
import pandas as pd
import skimage.io

filezNOW = ["composite_transposed_['male212']_['B']_['male'].tif",
"composite_transposed_['male212']_['C']_['male'].tif",
"composite_transposed_['male212']_['G']_['male'].tif",
 "composite_transposed_['male212']_['D']_['male'].tif",
 "composite_transposed_['male212']_['E']_['male'].tif", # F is missing
 ]
sectionsNOW = [96.0, 97.0, 98.0, 99.0, 100.0]

dfs = []

for it, yyy in zip(filezNOW, sectionsNOW):
    yyy = str(int(float(yyy)))
    print(yyy)

    try:

        image = skimage.io.imread(f"/data/luca/lipidatlas/ManuscriptAnalysisRound3/annotation/male212_exported/"+it+"_Coords.tif") 
        height, width, _ = image.shape
        x_coords, y_coords = np.meshgrid(np.arange(width), np.arange(height))
        x_coords_flat = x_coords.flatten()
        y_coords_flat = y_coords.flatten()
        flattened_array = image.reshape(-1, 3)

        df = pd.DataFrame(flattened_array, columns=['xccf', 'yccf', 'zccf'])

        df['x_pos'] = x_coords_flat
        df['y_pos'] = y_coords_flat

        df.index = "section"+yyy+"_pixel"+(df['y_pos']+1).astype(str)+"_"+(df['x_pos']+1).astype(str)
        df.drop(columns=['x_pos', 'y_pos'], inplace=True)

        dfs.append(df)

    except:
        print("issue with: " + it)

this_ccf = pd.concat(dfs)

indexes = np.array(this_ccf.index)
pixels_index_set = set(pixelmet.index)
goodindexes = [ind for ind in indexes if ind in pixels_index_set]
this_ccf = this_ccf.loc[goodindexes,:]
this_ccf['x_index'] = (this_ccf['xccf']*40).astype(int)
this_ccf['y_index'] = (this_ccf['yccf']*40).astype(int)
this_ccf['z_index'] = (this_ccf['zccf']*40).astype(int)
print(this_ccf)
ccfs.append(this_ccf)

In [ ]:
# for the males

import re
import os
import skimage
import numpy as np
import pandas as pd
import skimage.io

filezNOW = ["composite_transposed_['male203']_['B']_['male'].tif",
 "composite_transposed_['male203']_['C']_['male'].tif",
 "composite_transposed_['male203']_['A']_['male'].tif",
 "composite_transposed_['male203']_['D']_['male'].tif",
 "composite_transposed_['male203']_['E']_['male'].tif",
"composite_transposed_['male203']_['F']_['male'].tif"]
sectionsNOW = [90.0, 91.0, 92.0, 93.0, 94.0, 95.0]

dfs = []

for it, yyy in zip(filezNOW, sectionsNOW):
    yyy = str(int(float(yyy)))
    print(yyy)

    try:

        image = skimage.io.imread(f"/data/luca/lipidatlas/ManuscriptAnalysisRound3/annotation/exported_male_203/"+it+"_Coords.tif") 
        height, width, _ = image.shape
        x_coords, y_coords = np.meshgrid(np.arange(width), np.arange(height))
        x_coords_flat = x_coords.flatten()
        y_coords_flat = y_coords.flatten()
        flattened_array = image.reshape(-1, 3)

        df = pd.DataFrame(flattened_array, columns=['xccf', 'yccf', 'zccf'])

        df['x_pos'] = x_coords_flat
        df['y_pos'] = y_coords_flat

        df.index = "section"+yyy+"_pixel"+(df['y_pos']+1).astype(str)+"_"+(df['x_pos']+1).astype(str)
        df.drop(columns=['x_pos', 'y_pos'], inplace=True)

        dfs.append(df)

    except:
        print("issue with: " + it)

this_ccf = pd.concat(dfs)

indexes = np.array(this_ccf.index)
pixels_index_set = set(pixelmet.index)
goodindexes = [ind for ind in indexes if ind in pixels_index_set]
this_ccf = this_ccf.loc[goodindexes,:]
this_ccf['x_index'] = (this_ccf['xccf']*40).astype(int)
this_ccf['y_index'] = (this_ccf['yccf']*40).astype(int)
this_ccf['z_index'] = (this_ccf['zccf']*40).astype(int)
print(this_ccf)
ccfs.append(this_ccf)

In [ ]:
# for the males

import re
import os
import skimage
import numpy as np
import pandas as pd
import skimage.io

filezNOW = ["composite_transposed_['male305']_['1A']_['male'].tif",
"composite_transposed_['male305']_['1B']_['male'].tif",
"composite_transposed_['male305']_['1C']_['male'].tif",
"composite_transposed_['male305']_['3D']_['male'].tif",
"composite_transposed_['male305']_['1E']_['male'].tif",
"composite_transposed_['male305']_['1F']_['male'].tif"]
sectionsNOW = [102.0, 103.0, 104.0, 105.0, 106.0, 107.0]

dfs = []

for it, yyy in zip(filezNOW, sectionsNOW):
    yyy = str(int(float(yyy)))
    print(yyy)

    try:

        image = skimage.io.imread(f"/data/luca/lipidatlas/ManuscriptAnalysisRound3/annotation/male305_exported/"+it+"_Coords.tif") 
        height, width, _ = image.shape
        x_coords, y_coords = np.meshgrid(np.arange(width), np.arange(height))
        x_coords_flat = x_coords.flatten()
        y_coords_flat = y_coords.flatten()
        flattened_array = image.reshape(-1, 3)

        df = pd.DataFrame(flattened_array, columns=['xccf', 'yccf', 'zccf'])

        df['x_pos'] = x_coords_flat
        df['y_pos'] = y_coords_flat

        df.index = "section"+yyy+"_pixel"+(df['y_pos']+1).astype(str)+"_"+(df['x_pos']+1).astype(str)
        df.drop(columns=['x_pos', 'y_pos'], inplace=True)

        dfs.append(df)

    except:
        print("issue with: " + it)

this_ccf = pd.concat(dfs)

indexes = np.array(this_ccf.index)
pixels_index_set = set(pixelmet.index)
goodindexes = [ind for ind in indexes if ind in pixels_index_set]
this_ccf = this_ccf.loc[goodindexes,:]
this_ccf['x_index'] = (this_ccf['xccf']*40).astype(int)
this_ccf['y_index'] = (this_ccf['yccf']*40).astype(int)
this_ccf['z_index'] = (this_ccf['zccf']*40).astype(int)
print(this_ccf)
ccfs.append(this_ccf)

In [ ]:
# for GBA

import re
import os
import skimage
import numpy as np
import pandas as pd
import skimage.io

filezNOW = np.sort(os.listdir("/data/luca/lipidatlas/ManuscriptAnalysisRound3/annotation/ABBA-GBA"))
print(filezNOW)
sectionsNOW = [126.0,127.0,128.0,129.0,130.0,131.0,132.0]

dfs = []

for it, yyy in zip(filezNOW, sectionsNOW):
    yyy = str(int(float(yyy)))
    print(yyy)

    try:

        image = skimage.io.imread(f"/data/luca/lipidatlas/ManuscriptAnalysisRound3/annotation/GBA_exported/"+it+"_Coords.tif")
        height, width, _ = image.shape
        x_coords, y_coords = np.meshgrid(np.arange(width), np.arange(height))
        x_coords_flat = x_coords.flatten()
        y_coords_flat = y_coords.flatten()
        flattened_array = image.reshape(-1, 3)

        df = pd.DataFrame(flattened_array, columns=['xccf', 'yccf', 'zccf'])

        df['x_pos'] = x_coords_flat
        df['y_pos'] = y_coords_flat

        df.index = "section"+yyy+"_pixel"+(df['y_pos']+1).astype(str)+"_"+(df['x_pos']+1).astype(str)
        df.drop(columns=['x_pos', 'y_pos'], inplace=True)

        dfs.append(df)

    except:
        print("issue with: " + it)

this_ccf = pd.concat(dfs)

indexes = np.array(this_ccf.index)
pixels_index_set = set(pixelmet.index)
goodindexes = [ind for ind in indexes if ind in pixels_index_set]
this_ccf = this_ccf.loc[goodindexes,:]
this_ccf['x_index'] = (this_ccf['xccf']*40).astype(int)
this_ccf['y_index'] = (this_ccf['yccf']*40).astype(int)
this_ccf['z_index'] = (this_ccf['zccf']*40).astype(int)
print(this_ccf)
ccfs.append(this_ccf)

In [ ]:
len(ccfs)

In [ ]:
ccfsnew = pd.concat(ccfs, axis=0)
ccfsnew

In [ ]:
from bg_atlasapi import BrainGlobeAtlas

atlas = BrainGlobeAtlas("allen_mouse_25um")
reference_image = atlas.reference

erodedannot = np.load("eroded_annot.npy")
erodedannot.shape

nan_mask = ccfsnew[['x_index', 'y_index', 'z_index']].isna().any(axis=1)

ccfsnew['boundary'] = np.nan
valid_ccfsnew = ccfsnew.loc[~nan_mask, ['x_index', 'y_index', 'z_index']]
valid_indices = valid_ccfsnew.astype(int)
X_MAX, Y_MAX, Z_MAX = erodedannot.shape

in_bounds_mask = (
    (valid_indices['x_index'] >= 0) & (valid_indices['x_index'] < X_MAX) &
    (valid_indices['y_index'] >= 0) & (valid_indices['y_index'] < Y_MAX) &
    (valid_indices['z_index'] >= 0) & (valid_indices['z_index'] < Z_MAX)
)

valid_and_inbounds_indices = valid_indices.loc[in_bounds_mask]

ccfsnew.loc[valid_and_inbounds_indices.index, 'boundary'] = erodedannot[
    valid_and_inbounds_indices['x_index'].values,
    valid_and_inbounds_indices['y_index'].values,
    valid_and_inbounds_indices['z_index'].values
]

#### assigning Allen regions

from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
import pandas as pd
import numpy as np
from tqdm import tqdm

## use with care!
import warnings
warnings.filterwarnings('ignore')

mcc = MouseConnectivityCache(manifest_file='mouse_connectivity_manifest.json')
structure_tree = mcc.get_structure_tree()

data = ccfsnew
annotation, _ = mcc.get_annotation_volume()

import pandas as pd
import matplotlib.pyplot as plt

num_rows = len(data)
data['acronym'] = [''] * num_rows
data['id'] = [0] * num_rows
data['name'] = [''] * num_rows
data['structure_id_path'] = [''] * num_rows
data['structure_set_ids'] = [''] * num_rows
data['rgb_triplet'] = [''] * num_rows

for xxx in tqdm(range(0, data.shape[0])):
    row = data.iloc[xxx,:]
    try:
        x_index = int(row['x_index'])
        y_index = int(row['y_index'])
        z_index = int(row['z_index'])
    
        if 0 <= x_index < annotation.shape[0] and 0 <= y_index < annotation.shape[1] and 0 <= z_index < annotation.shape[2]:
            index = annotation[x_index, y_index, z_index]
            brain_region = structure_tree.get_structures_by_id([index])[0]

            if brain_region is not None:

                data['acronym'][xxx] = brain_region['acronym']
                data['id'][xxx] = brain_region['id']
                data['name'][xxx] = brain_region['name']
                data['structure_id_path'][xxx] = brain_region['structure_id_path']
                data['structure_set_ids'][xxx] = brain_region['structure_set_ids']
                data['rgb_triplet'][xxx] = brain_region['rgb_triplet']
    except:
        continue
            

def rgb_to_hex_safe(rgb):
    try:
        return '#{:02x}{:02x}{:02x}'.format(*rgb)
    except (TypeError, IndexError):
        return "#000000"

data['allencolor'] = data['rgb_triplet'].apply(rgb_to_hex_safe)

data

In [ ]:
data.to_hdf("allenotherbrains_full.h5ad", key="table")

In [ ]:
for xxx in range(74, 132):
    try:
        
        tmp = data.loc[pixelmet['SectionID'] == xxx,:]
        
        print(xxx)

        plt.scatter(tmp['zccf'], -tmp['yccf'], c=tmp['allencolor'])
        plt.show()
    except:
        continue

In [ ]:
metadata.loc[metadata['SectionID'].isin([87,101,126,130]), ['Section', 'Sample']]